In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!ls -l '/content/drive/My Drive/Colab Notebooks/datasets/final_split.zip'

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/datasets/final_split.zip

In [ ]:
!ls -l final

In [ ]:
train_dir = 'final/train'
test_dir = 'final/test'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(480,480), batch_size=20, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(480,480), batch_size=20, class_mode='binary')

In [ ]:
from keras.applications import  ResNet50

conv_base =  ResNet50(weights='imagenet', include_top=False, input_shape =(480,480,3))
conv_base.summary()

In [ ]:
set_trainable = False

for layer in conv_base.layers:
  if layer.name =='conv5_block3_2_conv' :      
    set_trainable = True

  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

conv_base.summary()



In [ ]:
from keras import layers, models, optimizers

model = models.Sequential()
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary()

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.000005), metrics=['accuracy'])    

In [ ]:
%%time

Hist = model.fit(train_generator, steps_per_epoch=21,
                epochs=50, validation_data=test_generator, validation_steps=5) 

In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=5)
loss, accuracy

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['loss'])
plt.plot(epochs, Hist.history['val_loss'])

plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()


plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['accuracy'])
plt.plot(epochs, Hist.history['val_accuracy'])

plt.legend(['Training accuracy', 'Validation accuracy'])
plt.grid()
plt.show()